In [1]:
import json
# read api key from tally_api_key.json
TALLY_API_KEY = json.load(open('tally_api_key.json'))['key']

In [2]:
import requests

url = "https://api.tally.xyz/query"

headers = {
  'Api-Key': TALLY_API_KEY,
  'Content-Type': 'application/json'
}

In [3]:
payload="{\"query\":\"query Chains {\\n    chains {\\n        id\\n    }\\n}\",\"variables\":{}}"

chains_response = requests.request("POST", url, headers=headers, data=payload)

chains_data = chains_response.json()
chains_data

{'data': {'chains': [{'id': 'eip155:42170'},
   {'id': 'eip155:4'},
   {'id': 'eip155:97'},
   {'id': 'eip155:42161'},
   {'id': 'eip155:43113'},
   {'id': 'eip155:42'},
   {'id': 'eip155:534353'},
   {'id': 'eip155:1'},
   {'id': 'eip155:100'},
   {'id': 'eip155:10'},
   {'id': 'eip155:421611'},
   {'id': 'eip155:43114'},
   {'id': 'eip155:5'},
   {'id': 'eip155:80001'},
   {'id': 'eip155:69'},
   {'id': 'eip155:11155111'},
   {'id': 'eip155:137'},
   {'id': 'eip155:421613'},
   {'id': 'eip155:412346'},
   {'id': 'eip155:84531'},
   {'id': 'eip155:56'},
   {'id': 'eip155:420'},
   {'id': 'eip155:1284'}]}}

In [4]:
chains = chains_data['data']['chains']
chains = [c['id'] for c in chains]
chains

['eip155:42170',
 'eip155:4',
 'eip155:97',
 'eip155:42161',
 'eip155:43113',
 'eip155:42',
 'eip155:534353',
 'eip155:1',
 'eip155:100',
 'eip155:10',
 'eip155:421611',
 'eip155:43114',
 'eip155:5',
 'eip155:80001',
 'eip155:69',
 'eip155:11155111',
 'eip155:137',
 'eip155:421613',
 'eip155:412346',
 'eip155:84531',
 'eip155:56',
 'eip155:420',
 'eip155:1284']

In [5]:
def make_proposals_query(chain: str) -> str:
    query = """query Proposals {
        proposals(
            chainId: "%s"
        ) {
            platform_deployment_id: governanceId
            proposal_id: id
            author: proposer {
                id
            }
            date: start {
                timestamp
            }
            voteStats {
                support
                votes
            }
        }
    }"""
    return query % chain

In [6]:
import time

responses_by_chain = []
for chain in chains:
    status_code = 999
    while status_code != 200:
        query = make_proposals_query(chain)
        response = requests.request("POST", url, headers=headers, data=json.dumps({'query': query}))
        status_code = response.status_code
        if status_code != 200:
            print('error', chain, response.status_code)
            # wait
            time.sleep(5)
    responses_by_chain.append(response.json())
    print('done', chain, response.status_code)
    time.sleep(2)

done eip155:42170 200
done eip155:4 200
done eip155:97 200
done eip155:42161 200
done eip155:43113 200
done eip155:42 200
done eip155:534353 200
done eip155:1 200
done eip155:100 200
done eip155:10 200
done eip155:421611 200
done eip155:43114 200
done eip155:5 200
done eip155:80001 200
done eip155:69 200
done eip155:11155111 200
done eip155:137 200
done eip155:421613 200
done eip155:412346 200
done eip155:84531 200
done eip155:56 200
done eip155:420 200
done eip155:1284 200


In [7]:
import pandas as pd
# make a dataframe of all proposals
proposals = []
for response in responses_by_chain:
    proposals += response['data']['proposals']
df = pd.DataFrame(proposals)
df

,platform_deployment_id,proposal_id,author,date,voteStats
0,eip155:42170:0x4e68c0f354c03bEbf2a66E1f4fC1a0e...,2469566067493070112382155014139101658401666661...,{'id': 'eip155:42170:0x1A320E53A25f518B893F286...,{'timestamp': '2023-06-28T02:00:47Z'},"[{'support': 'FOR', 'votes': '0'}, {'support':..."
1,eip155:42170:0x4e68c0f354c03bEbf2a66E1f4fC1a0e...,3687053733533920144921907414590262232442515184...,{'id': 'eip155:42170:0x1A320E53A25f518B893F286...,{'timestamp': '2023-06-28T20:31:23Z'},"[{'support': 'FOR', 'votes': '1'}, {'support':..."
2,eip155:4:0x1874306110B9cd9CcE56Ba5748cb4f78DCA...,2374853609165189405470451842531233611721691995...,{'id': 'eip155:4:0x2a7ac99a7cE7777dD4598e8eb89...,{'timestamp': '2022-07-18T12:21:52Z'},"[{'support': 'FOR', 'votes': '1'}, {'support':..."
3,eip155:4:0xE108cdBc1eE0F775D93C12C071a1F97d94A...,1037135589049921000232560869388807622276297882...,{'id': 'eip155:4:0x73cCB30FA3715FFad5ceC6d5533...,{'timestamp': '2022-07-28T22:20:47Z'},"[{'support': 'FOR', 'votes': '0'}, {'support':..."
4,eip155:4:0xB09525419AfF0c7e28c07597DA9EdEe2BdF...,1,{'id': 'eip155:4:0xb1e020029EBAe05673Fc9166E12...,{'timestamp': '2022-08-12T19:32:54Z'},"[{'support': 'FOR', 'votes': '0'}, {'support':..."
...,...,...,...,...,...
7877,eip155:1284:0xfc4DFB17101A12C5CEc5eeDd8E92B5b1...,29,{'id': 'eip155:1284:0x84165355AFbFB553f3a58f1C...,{'timestamp': '2077-01-07T00:34:38Z'},"[{'support': 'FOR', 'votes': '0'}, {'support':..."
7878,eip155:1284:0xfc4DFB17101A12C5CEc5eeDd8E92B5b1...,30,{'id': 'eip155:1284:0x155475353D0a4B0e64AB42C1...,{'timestamp': '2077-03-29T05:35:50Z'},"[{'support': 'FOR', 'votes': '0'}, {'support':..."
7879,eip155:1284:0x311346FDe706FEBAB09cCEd059A05566...,2170557268702905712866614377830003373598010142...,{'id': 'eip155:1284:0x73ae2354A270a6AFF6F84Ce8...,{'timestamp': '2023-05-07T14:08:42Z'},"[{'support': 'FOR', 'votes': '17'}, {'support'..."
7880,eip155:1284:0xfc4DFB17101A12C5CEc5eeDd8E92B5b1...,1,{'id': 'eip155:1284:0xb2F5a22Bd154924402cf31a9...,{'timestamp': '2069-06-15T03:41:50Z'},"[{'support': 'FOR', 'votes': '0'}, {'support':..."


In [8]:
# replace author column with author_id
df['author'] = df['author'].apply(lambda x: x['id'])
df.head()

,platform_deployment_id,proposal_id,author,date,voteStats
0,eip155:42170:0x4e68c0f354c03bEbf2a66E1f4fC1a0e...,2469566067493070112382155014139101658401666661...,eip155:42170:0x1A320E53A25f518B893F286f3600cc2...,{'timestamp': '2023-06-28T02:00:47Z'},"[{'support': 'FOR', 'votes': '0'}, {'support':..."
1,eip155:42170:0x4e68c0f354c03bEbf2a66E1f4fC1a0e...,3687053733533920144921907414590262232442515184...,eip155:42170:0x1A320E53A25f518B893F286f3600cc2...,{'timestamp': '2023-06-28T20:31:23Z'},"[{'support': 'FOR', 'votes': '1'}, {'support':..."
2,eip155:4:0x1874306110B9cd9CcE56Ba5748cb4f78DCA...,2374853609165189405470451842531233611721691995...,eip155:4:0x2a7ac99a7cE7777dD4598e8eb898862a244...,{'timestamp': '2022-07-18T12:21:52Z'},"[{'support': 'FOR', 'votes': '1'}, {'support':..."
3,eip155:4:0xE108cdBc1eE0F775D93C12C071a1F97d94A...,1037135589049921000232560869388807622276297882...,eip155:4:0x73cCB30FA3715FFad5ceC6d55333c94211C...,{'timestamp': '2022-07-28T22:20:47Z'},"[{'support': 'FOR', 'votes': '0'}, {'support':..."
4,eip155:4:0xB09525419AfF0c7e28c07597DA9EdEe2BdF...,1,eip155:4:0xb1e020029EBAe05673Fc9166E12A8FC603d...,{'timestamp': '2022-08-12T19:32:54Z'},"[{'support': 'FOR', 'votes': '0'}, {'support':..."


In [9]:
# read date.timestamp as date
df['date'] = df['date'].apply(lambda x: x['timestamp'])
# convert date to datetime from 2022-09-12T06:49:26Z format
df['date'] = pd.to_datetime(df['date'])
df.head()

,platform_deployment_id,proposal_id,author,date,voteStats
0,eip155:42170:0x4e68c0f354c03bEbf2a66E1f4fC1a0e...,2469566067493070112382155014139101658401666661...,eip155:42170:0x1A320E53A25f518B893F286f3600cc2...,2023-06-28 02:00:47+00:00,"[{'support': 'FOR', 'votes': '0'}, {'support':..."
1,eip155:42170:0x4e68c0f354c03bEbf2a66E1f4fC1a0e...,3687053733533920144921907414590262232442515184...,eip155:42170:0x1A320E53A25f518B893F286f3600cc2...,2023-06-28 20:31:23+00:00,"[{'support': 'FOR', 'votes': '1'}, {'support':..."
2,eip155:4:0x1874306110B9cd9CcE56Ba5748cb4f78DCA...,2374853609165189405470451842531233611721691995...,eip155:4:0x2a7ac99a7cE7777dD4598e8eb898862a244...,2022-07-18 12:21:52+00:00,"[{'support': 'FOR', 'votes': '1'}, {'support':..."
3,eip155:4:0xE108cdBc1eE0F775D93C12C071a1F97d94A...,1037135589049921000232560869388807622276297882...,eip155:4:0x73cCB30FA3715FFad5ceC6d55333c94211C...,2022-07-28 22:20:47+00:00,"[{'support': 'FOR', 'votes': '0'}, {'support':..."
4,eip155:4:0xB09525419AfF0c7e28c07597DA9EdEe2BdF...,1,eip155:4:0xb1e020029EBAe05673Fc9166E12A8FC603d...,2022-08-12 19:32:54+00:00,"[{'support': 'FOR', 'votes': '0'}, {'support':..."


In [10]:
# remove tz info from date
df['date'] = df['date'].apply(lambda x: x.replace(tzinfo=None))
df.head()

,platform_deployment_id,proposal_id,author,date,voteStats
0,eip155:42170:0x4e68c0f354c03bEbf2a66E1f4fC1a0e...,2469566067493070112382155014139101658401666661...,eip155:42170:0x1A320E53A25f518B893F286f3600cc2...,2023-06-28 02:00:47,"[{'support': 'FOR', 'votes': '0'}, {'support':..."
1,eip155:42170:0x4e68c0f354c03bEbf2a66E1f4fC1a0e...,3687053733533920144921907414590262232442515184...,eip155:42170:0x1A320E53A25f518B893F286f3600cc2...,2023-06-28 20:31:23,"[{'support': 'FOR', 'votes': '1'}, {'support':..."
2,eip155:4:0x1874306110B9cd9CcE56Ba5748cb4f78DCA...,2374853609165189405470451842531233611721691995...,eip155:4:0x2a7ac99a7cE7777dD4598e8eb898862a244...,2022-07-18 12:21:52,"[{'support': 'FOR', 'votes': '1'}, {'support':..."
3,eip155:4:0xE108cdBc1eE0F775D93C12C071a1F97d94A...,1037135589049921000232560869388807622276297882...,eip155:4:0x73cCB30FA3715FFad5ceC6d55333c94211C...,2022-07-28 22:20:47,"[{'support': 'FOR', 'votes': '0'}, {'support':..."
4,eip155:4:0xB09525419AfF0c7e28c07597DA9EdEe2BdF...,1,eip155:4:0xb1e020029EBAe05673Fc9166E12A8FC603d...,2022-08-12 19:32:54,"[{'support': 'FOR', 'votes': '0'}, {'support':..."


In [11]:
# sum the 'votes' value for each item in voteStats and save as votes_count
df['votes_count'] = df['voteStats'].apply(lambda x: sum([int(i['votes']) for i in x]))
# drop voteStats column
df.drop('voteStats', axis=1, inplace=True)
df.head()

,platform_deployment_id,proposal_id,author,date,votes_count
0,eip155:42170:0x4e68c0f354c03bEbf2a66E1f4fC1a0e...,2469566067493070112382155014139101658401666661...,eip155:42170:0x1A320E53A25f518B893F286f3600cc2...,2023-06-28 02:00:47,0
1,eip155:42170:0x4e68c0f354c03bEbf2a66E1f4fC1a0e...,3687053733533920144921907414590262232442515184...,eip155:42170:0x1A320E53A25f518B893F286f3600cc2...,2023-06-28 20:31:23,1
2,eip155:4:0x1874306110B9cd9CcE56Ba5748cb4f78DCA...,2374853609165189405470451842531233611721691995...,eip155:4:0x2a7ac99a7cE7777dD4598e8eb898862a244...,2022-07-18 12:21:52,1
3,eip155:4:0xE108cdBc1eE0F775D93C12C071a1F97d94A...,1037135589049921000232560869388807622276297882...,eip155:4:0x73cCB30FA3715FFad5ceC6d55333c94211C...,2022-07-28 22:20:47,0
4,eip155:4:0xB09525419AfF0c7e28c07597DA9EdEe2BdF...,1,eip155:4:0xb1e020029EBAe05673Fc9166E12A8FC603d...,2022-08-12 19:32:54,0


In [12]:
df['platform'] = 'tally'
# reorder cols so that platform is first
df = df[['platform', 'platform_deployment_id', 'proposal_id', 'author', 'date', 'votes_count']]
df.head()

,platform,platform_deployment_id,proposal_id,author,date,votes_count
0,tally,eip155:42170:0x4e68c0f354c03bEbf2a66E1f4fC1a0e...,2469566067493070112382155014139101658401666661...,eip155:42170:0x1A320E53A25f518B893F286f3600cc2...,2023-06-28 02:00:47,0
1,tally,eip155:42170:0x4e68c0f354c03bEbf2a66E1f4fC1a0e...,3687053733533920144921907414590262232442515184...,eip155:42170:0x1A320E53A25f518B893F286f3600cc2...,2023-06-28 20:31:23,1
2,tally,eip155:4:0x1874306110B9cd9CcE56Ba5748cb4f78DCA...,2374853609165189405470451842531233611721691995...,eip155:4:0x2a7ac99a7cE7777dD4598e8eb898862a244...,2022-07-18 12:21:52,1
3,tally,eip155:4:0xE108cdBc1eE0F775D93C12C071a1F97d94A...,1037135589049921000232560869388807622276297882...,eip155:4:0x73cCB30FA3715FFad5ceC6d55333c94211C...,2022-07-28 22:20:47,0
4,tally,eip155:4:0xB09525419AfF0c7e28c07597DA9EdEe2BdF...,1,eip155:4:0xb1e020029EBAe05673Fc9166E12A8FC603d...,2022-08-12 19:32:54,0


In [13]:
# save as csv
df.to_csv('tally_proposals.csv', index=False)